In [2]:
import pandas as pd
import numpy as np
from y_engineering import pct_log
from Indicators import *

In [40]:
import pathlib
import datetime

In [41]:
pathlib.Path().resolve()

PosixPath('/home/jack/Desktop/Programming/tr_bot')

In [42]:
tickers = ["AAPL", "AMZN", "GOOG", "META", "MSFT", "NVDA"]

In [43]:
# getting raw data
data = {}
for ticker in tickers:
    raw_path = "/home/jack/Desktop/Programming/tr_bot/data/raw/" + ticker + ".csv"
    data[ticker] = pd.read_csv(raw_path)

In [44]:
#saving data split into train and test files with extra features

for key, df in data.items():
    #adding basic features
    df["pct"] = pct_log(df, "Adj Close", log=False)
    df["pct_log"] = pct_log(df, "Adj Close")
    
    df["ema_20"] = ema(df, period=20, column='Adj Close')
    df["ema_50"] = ema(df, period=50, column='Adj Close')
    df["ema_200"] = ema(df, period=200, column='Adj Close')
    df["ema_pct_20"] = pct_log(df, y_col='ema_20', time_to_pred = 1, pct=True, log=False)
    df["ema_pct_50"] = pct_log(df, y_col='ema_50', time_to_pred = 1, pct=True, log=False)
    df["ema_pct_200"] = pct_log(df, y_col='ema_200', time_to_pred = 1, pct=True, log=False)
    
    df["rsi_20"] = rsi(df, period=20, column='Adj Close')
    df["rsi_20"] = (df["rsi_20"]-50)/100
    
    df['y_binary_sma4'] = (sma(df, period=4, column='Adj Close').shift(periods=-4) > df['Adj Close']).astype(int)
    
    #saving data
    path = "/home/jack/Desktop/Programming/tr_bot/data"
    df[:-500].to_csv(path + "/train/" + key + "_train.csv", index=False)
    df[-500:].to_csv(path + "/test/" + key + "_test.csv", index=False)